In [ ]:
import os
os.chdir("..")

In [ ]:
from src.data.data_process import DataReg
import arviz as az
import bambi as bmb
import matplotlib.pyplot as plt
from linearmodels.panel import RandomEffects, PanelOLS
import polars as pl
import numpy as np
dr = DataReg()

In [ ]:
dr.conn.sql("SELECT * FROM qcewtable WHERE ein=='660567825'").pl() #INFO We cound manualy obtain if they are foreing

In [ ]:
dr = DataReg()

df_qcew = dr.regular_data().filter(pl.col("year")>=2012)
df_qcew = df_qcew.filter(pl.col("ein") != "")
pr_zips = dr.spatial_data()["zipcode"].to_list()

df_qcew = df_qcew.with_columns(
    foreign=pl.when(pl.col("ui_addr_5_zip").is_in(pr_zips)).then(0).otherwise(1)
)

df_qcew = df_qcew.group_by(["year", "qtr", "zipcode", "ein"]).agg(
        total_employment=pl.col("total_employment").sum(),
        total_wages=pl.col("total_wages").sum(),
        foreign=pl.col("foreign").mean()
        )
df_qcew = df_qcew.filter((pl.col("foreign") == 1) | (pl.col("foreign") == 0))

# df_qcew = df_qcew.filter(pl.col("year")>= 2020)

tmp = pl.from_pandas(dr.make_spatial_dataset().drop("geometry", axis=1))

master = df_qcew.join(tmp,on=["year","qtr","zipcode"], how="inner", validate="m:1")

# List of columns that we want to exclude
exclude_columns = ["year", "ein", "zipcode", "qtr", "state"]

# Get all columns except the excluded ones
columns_to_transform = [col for col in master.columns if col not in exclude_columns]

# Create a new list of expressions to add log-transformed columns
log_columns = [
    (pl.col(col).log().alias(f"log_{col}")) for col in columns_to_transform
]

# Add the log-transformed columns to the DataFrame
master = master.with_columns(log_columns)
master = master.with_columns(
    k_dummy=pl.col("foreign")* pl.col("log_k_index")
)

data = master.to_pandas().copy()
        
data["date2"] = data["year"] * 10 + data["qtr"] 
data['date'] = data['date2'].astype('category')
data["main_id"] = data['zipcode'] + "-" + data['ein']
data['zipcode'] = data['zipcode'].astype('category')
data['ein'] = data['ein'].astype('category')
data["main_id"] = data['main_id'].astype('category')
data = data.sort_values(["year","qtr","ein"]).reset_index(drop=True)
data

In [ ]:
data[data["commute_car"] == 0]["zipcode"].unique()

- rain by naics (2 digits)
- drop null zipcodes
- broke down the 72 (remove food services as its own sector)
- only the first 2 regression(0,1)


In [ ]:
data_pr = data[data["foreign"] == 0]
data_panel = data_pr.set_index(["main_id","date2"])
# data_panel = data_panel.drop("geometry", axis=1)
model = PanelOLS.from_formula(
    'log_total_employment ~ log_k_index + own_children6 + own_children17 + commute_car + food_stamp + with_social_security + EntityEffects',
    data=data_panel
)
results = model.fit(cov_type='clustered', cluster_entity=True)
print(results.summary)

In [ ]:
data_pr = data[data["foreign"] == 1]
data_panel = data_pr.set_index(["main_id","date2"])
# data_panel = data_panel.drop("geometry", axis=1)
model = PanelOLS.from_formula(
    'log_total_employment ~ log_k_index + own_children6 + own_children17 + commute_car + food_stamp + with_social_security + EntityEffects',
    data=data_panel
)
results = model.fit(cov_type='clustered', cluster_entity=True)
print(results.summary)

In [ ]:
data_pr = data
data_panel = data_pr.set_index(["main_id","date2"])
# data_panel = data_panel.drop("geometry", axis=1)
model = PanelOLS.from_formula(
    'log_total_employment ~ log_k_index + k_dummy + own_children6 + own_children17 + commute_car + food_stamp + with_social_security + EntityEffects',
    data=data_panel
)
results = model.fit(cov_type='clustered', cluster_entity=True)
print(results.summary)

In [ ]:
data_pr = data[data["foreign"] == 0]
model = bmb.Model(
    "total_employment ~ 0 + date + ein + k_index + own_children6 + own_children17 + commute_car + food_stamp + with_social_security",
    data_pr, dropna=True,
)
results = model.fit(sample_kwargs={"nuts_sampler" :"blackjax"},cores=15)



In [ ]:
az.plot_trace(
    results
)

In [ ]:


plt.savefig("data/results/trace_plot_pr.png", format="png")

res = az.summary(results)
res.to_csv("data/results/results_pr.csv")

In [ ]:
data_pr = data[data["foreign"] == 1]
model = bmb.Model(
    "total_employment ~ 0 + k_index + own_children6 + own_children17  + commute_car + food_stamp + with_social_security",
    data_pr, dropna=True,
)
results = model.fit(sample_kwargs={"nuts_sampler" :"blackjax"},cores=15)

az.plot_trace(
    results
)

plt.savefig("data/results/trace_plot_foreing.png", format="png")

res = az.summary(results)
res.to_csv("data/results/results_foreing.csv")

In [ ]:
model = bmb.Model(
    "total_employment ~ 0 + k_index + k_dummy + own_children6 + own_children17  + commute_car + food_stamp + with_social_security",
    data, dropna=True,
)
results = model.fit(sample_kwargs={"nuts_sampler" :"blackjax"},cores=15)

az.plot_trace(
    results
)

plt.savefig("data/results/trace_plot_full.png", format="png")

res = az.summary(results)
res.to_csv("data/results/results_full.csv")